In [1]:
import numpy as np
from hmmlearn import hmm


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.1,0.9],[0.9,0.1]])
model.transmat_ = np.array([[0.9,0.1],[0.1,0.9]])
model.startprob_ = np.array([0.8,0.2])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [3]:
model2 = hmm.CategoricalHMM(n_components=hmm_states,n_iter=1000,random_state=30,implementation='scaling')
model2.fit(data.reshape(-1,1))
hidden_states = model2.predict(data.reshape(-1,1))
# print("Most likely hidden states:", hidden_states)


<bound method BaseHMM._fit_scaling of CategoricalHMM(implementation='scaling', n_components=2, n_features=2,
               n_iter=1000, random_state=RandomState(MT19937) at 0x130B24940)>
CategoricalHMM(implementation='scaling', n_components=2, n_features=2,
               n_iter=1000, random_state=RandomState(MT19937) at 0x130B24940)
I am in the _emission
[[0.62849892 0.37150108]
 [0.80204307 0.19795693]] [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]]
[[0.37150108 0.19795693]
 [0.37150108 0.19795693]
 [0.37150108 0.19795693]
 [0.37150108 0.19795693]
 [0.37150108 0.19795693]
 [0.37150108 0.19795693]
 [0.62849892 0.80204307]
 [0.37150108 0.19795693]
 [0.62849892 0.80204307]
 [0.62849892 0.80204307]]
<bound method BaseHMM._fit_scaling of CategoricalHMM(implementation='scaling', n_components=2, n_features=2,
               n_iter=1000, random_state=RandomState(MT19937) at 0x130B24940)>
CategoricalHMM(implementation='scaling', n_components=2, n_features=2,
               n_iter=1000, r

In [4]:
model2.emissionprob_

array([[0.08749693, 0.91250307],
       [0.88258569, 0.11741431]])

In [5]:
model2.transmat_

array([[0.88416888, 0.11583112],
       [0.11158945, 0.88841055]])

In [6]:
model2.monitor_.converged

True